# This notebook is for testing purposes , It is not the final deliverable.
## Contents:
0. [Message Encoding with message blocks](#0)
1. [Diffie-Hellman Key Exchange](#1)
2. [AlGamal](#2)
3. [Verifying Signature](#3)
4. [Handling Messages with AES](#4)
5. [Integration to chat.py](#5)

0. [Message Encoding with message blocks](#0)

### Import relevant libraries

In [2]:
from helpers.dh_gamal import deffie_hellman
from helpers.tools import from_file
import numpy as np
import hashlib

%reload_ext autoreload
%autoreload 2

### 1. [Diffie-Hellman Key Exchange](#1) (Done)

In [3]:
q_dh , a_dh , q_gamal , a_gamal = from_file("agree.txt")

In [4]:
np.random.seed(0)

X_a = np.random.randint(2, q_dh)
print("Alice Private Key for DH: ", X_a)
X_b = np.random.randint(2, q_dh)
print("Bob Private Key for DH: ", X_b)
Y_a = deffie_hellman(prime=q_dh, generator=a_dh, private_key=X_a)
print("Alice Public Key for DH: ", Y_a)
Y_b = deffie_hellman(prime=q_dh, generator=a_dh, private_key=X_b)
print("Bob Public Key for DH: ", Y_b)



Alice Private Key for DH:  14
Bob Private Key for DH:  17
Alice Public Key for DH:  6
Bob Public Key for DH:  10


In [5]:
np.random.seed(0)
X_a2 = np.random.randint(2,q_gamal)
print("Alice Private Key for Al-Gamal: ", X_a2)
X_b2 = np.random.randint(2, q_gamal)
print("Bob Private Key for Al-Gamal: ", X_b2)
Y_a2 = deffie_hellman(prime=q_gamal, generator=a_gamal, private_key=X_a2)
print("Alice Public Key for Al-Gamal: ", Y_a2)
Y_b2 = deffie_hellman(prime=q_gamal, generator=a_gamal, private_key=X_b2)
print("Bob Public Key for Al-Gamal: ", Y_b2)

Alice Private Key for Al-Gamal:  686
Bob Private Key for Al-Gamal:  561
Alice Public Key for Al-Gamal:  489
Bob Public Key for Al-Gamal:  731


### Al-Gamal without SHA1

In [6]:
def extended_euclidean(a,b):
    if a == 0:
        return b, 0, 1
    else:
        g, x, y = extended_euclidean(b % a, a)
        return g, y - (b // a) * x, x
    

def send_signature(public_dh,public_gamal,generator,prime):
    if public_dh >= prime:
        return None
    # generate random K 
    k = np.random.randint(2,prime-1)
    # calculate K 
    key = pow(public_gamal,k,prime)
    print("Key: ", key)
    c1 = pow(generator,k,prime)
    c2 = (key*public_dh) % prime
    return c1,c2
    

def verify_signature(public_dh,private_gamal,prime,C1,C2):
    key = pow(C1,private_gamal,prime)
    #print el key 
    print("Key: ", key)
    print(extended_euclidean(key,prime))
    if public_dh == (C2 * extended_euclidean(key,prime)[1]) % prime:
        return True
    return False
    

#### B using the public key of A

In [11]:
r,s = send_signature(Y_b,Y_a2,a_gamal,q_gamal)
# B will do the following : 
# will use Y_A2 as public key and the message as Y_A then generates c1 and c2
# then will send the signature to A
print("Signature: ", r,s)
# A will verify the signature by using the public key of B and the private key of A
# and the signature sent by B

verify_signature(Y_b,X_a2,q_gamal,r,s)

Key:  711
Signature:  199 261
Key:  711
(1, -137, 128)


True

#### A using the public key of B

In [7]:
r,s = send_signature(Y_b,Y_b2,a_gamal,q_gamal)
# B will do the following : 
# will use Y_A2 as public key and the message as Y_A then generates c1 and c2
# then will send the signature to A
print("Signature: ", r,s)
# A will verify the signature by using the public key of B and the private key of A
# and the signature sent by B

verify_signature(Y_b,X_b2,q_gamal,r,s)

Key:  129
Signature:  318 529
Key:  129
(1, 59, -10)


True

#### false signature 

In [8]:
r,s = send_signature(Y_b,Y_b2,a_gamal,q_gamal)
# B will do the following : 
# will use Y_A2 as public key and the message as Y_A then generates c1 and c2
# then will send the signature to A
print("Signature: ", r,s)
# A will verify the signature by using the public key of B and the private key of A
# and the signature sent by B

verify_signature(np.random.randint(200),X_b2,q_gamal,r,s)

Key:  228
Signature:  615 758
Key:  228
(1, -257, 77)


False

##### trying sha1

In [12]:
M = Y_a
m = int.from_bytes(hashlib.sha1(M.to_bytes(8)).digest())
m >> 154
# choose K 
K = np.random.randint(100)
print("K: ", K)
s1 = (a_gamal**K) % q_gamal
K_inv = extended_euclidean(K,q_gamal)[1]
s2 = (K_inv * (m - X_a2 * s1)) % (q_gamal-1)

print(s1,s2)

K:  21
639 470


In [15]:
#verifying
m = int.from_bytes(hashlib.sha1(M.to_bytes(8)).digest())
m >> 154

# get key from s1 
key = pow(s1,X_a2,q_gamal)
print(key)
print(extended_euclidean(key,q_gamal))
if pow(a_gamal,m,q_gamal) == pow(Y_a2,s1,q_gamal) * pow(s1,s2,q_gamal):
    print("Verified")


2
(1, -380, 1)


### 3. [Verifying signature](#3)

### 4. [Handling Messages](#4)

### 5. [Integration to chat.py](#5)